In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# zipfileをcsvにする
import zipfile
zf1 = zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip')
print(zf1.namelist())

zf2 = zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip')
print(zf2.namelist()) 

zf3 = zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip')
print(zf3.namelist()) 

In [ ]:
zf1.extractall()
zf2.extractall()
zf3.extractall()

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.describe

In [ ]:
# csv内のヘッダーを確認
import csv
csv_file = open("train.csv", "r", encoding="ms932", errors="", newline="" )
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
header = next(f)
print(header)

In [ ]:
# ダミー
# pd.get_dummies(train['type'])
pd.get_dummies(train['color'])
pd.get_dummies(test['color'])

training = pd.get_dummies(train, columns=['color'])
training
testing = pd.get_dummies(test, columns=['color'])
testing

In [ ]:
# ダミー導入後の相関
training.corr()

In [ ]:
import seaborn as sns
sns.stripplot(x='type',y='bone_length',data=train)

In [ ]:
import seaborn as sns
sns.stripplot(x='type',y='rotting_flesh',data=train)

In [ ]:
import seaborn as sns
sns.stripplot(x='type',y='hair_length',data=train)

In [ ]:
import seaborn as sns
sns.stripplot(x='type',y='has_soul',data=train)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.countplot(x = 'color', hue = "type", data = train)
# plt.legend(loc = "upper right", title = "Survived ~ Sibsp")

In [ ]:
# training['colors']=np.where((training['color_clear']+training['color_white'])>0, 0, 1)
# training.drop(['color_clear', 'color_white'], axis=1, inplace=True)
# training.head()

In [ ]:
# # 標準化
# df1 = training.iloc[:, 1:]
# left1 = df1.iloc[:, :4]
# right1 = df1.iloc[:, 5:]
# df1 = left1.join(right1)

# df2 = training.iloc[:, 5:]
# df3 = df2.iloc[:, :1]
# df3.head()

# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# ss_training = ss.fit_transform(df1[['bone_length', 'rotting_flesh', 'hair_length', 'has_soul','color_black','color_blood','color_blue','color_clear','color_green','color_white']])
# ss_trainings = pd.DataFrame(ss_training,columns=df1.columns[:])
# ss_trainings.head()


# df4 = df3.join(ss_trainings)
# df4.head()

In [ ]:
# df4.corr()

主成分分析

In [ ]:
# id と type を取り除く
df_training = training.iloc[:, 1:]
left = df_training.iloc[:, :4]
right = df_training.iloc[:, 5:]
df_training = left.join(right)
df_training.head()

In [ ]:
# 標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
trainings = ss.fit_transform(df_training)
trainings = pd.DataFrame(trainings, columns=df_training.columns[:])
trainings.head()

In [ ]:
# 主成分変換
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(trainings) #データを学習
feat = pca.transform(trainings) #データを成形
feat = pd.DataFrame(feat, columns = ['pc{}'.format(x+1) for x in range(len(trainings.columns))])
feat.head()

In [ ]:
# typeを数値化
training_c = pd.get_dummies(train, columns=['color'])
training_c['type'] = training_c['type'].astype('category').cat.codes
training_c.head()

# A = np.append(np.zeros(10),np.ones(11))
# 図
import matplotlib.pyplot as plt
plt.figure(figsize = (6,6))
plt.scatter(feat['pc1'], feat['pc2'], alpha = 0.8,c = training_c['type'])
# plt.scatter(feat['pc1'], feat['pc2'], alpha = 0.8, c=np.append(A,np.full(350,2)))
plt.grid()
plt.xlabel('pc1')
plt.ylabel('pc2')
# plt.legend('c')
plt.show()

ゴブリンの特徴を確認

In [ ]:
# ダミー
pd.get_dummies(train['type'])
pd.get_dummies(train['color'])
pd.get_dummies(test['color'])

training_gob = pd.get_dummies(train, columns=['color','type'])
training_gob

In [ ]:
# id と type を取り除く
df_training_gob = training_gob.iloc[:, 1:]
left_gob = df_training_gob.iloc[:, :4]
right_gob = df_training_gob.iloc[:, 5:10]
right_gob2 = df_training_gob.iloc[:,-1]
df_training_gob = left_gob.join(right_gob)
df_training_gob = df_training.join(right_gob2)
Gob = df_training_gob[df_training_gob['type_Goblin']!=0]
Gob.corr()

In [ ]:
# 標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
trainings_gob = ss.fit_transform(df_training_gob)
trainings_gob = pd.DataFrame(trainings_gob, columns=df_training_gob.columns[:])
trainings_gob.head()

In [ ]:
# 主成分変換
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(trainings_gob) #データを学習
feat_gob = pca.transform(trainings_gob) #データを成形
feat_gob = pd.DataFrame(feat_gob, columns = ['pc{}'.format(x+1) for x in range(len(trainings_gob.columns))])
feat_gob.head()

# A = np.append(np.zeros(10),np.ones(11))
# 図
import matplotlib.pyplot as plt
plt.figure(figsize = (6,6))
plt.scatter(feat_gob['pc1'], feat_gob['pc2'], alpha = 0.8,c = training_gob['type_Goblin'])
# plt.scatter(feat['pc1'], feat['pc2'], alpha = 0.8, c=np.append(A,np.full(350,2)))
plt.grid()
plt.xlabel('pc1')
plt.ylabel('pc2')
# plt.legend('c')
plt.show()

学習

In [ ]:
# ニューラルネットワーク
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import joblib

In [ ]:
# paramate
hidden_layer_sizes=(100,)
activation = 'relu'
solver = 'adam'
batch_size = 'auto'
alpha = 0.0001
random_state = 0
max_iter = 10000
# early_stopping = True

# 説明変数
# x = ['bone_length', 'rotting_flesh', 'hair_length', 'has_soul','color_black','color_blood','color_blue','color_clear','color_green','color_white']
x = ['bone_length', 'rotting_flesh', 'hair_length', 'has_soul']
# x = ['bone_length', 'rotting_flesh', 'hair_length', 'has_soul','color_clear','color_white']

train_X = training[x]

# 目的変数
y1 = ['type']
train_y1 = training[y1]

# 学習
clf = MLPClassifier(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation,
    solver=solver,
    batch_size=batch_size,
    alpha=alpha,
    random_state=random_state,
    max_iter=max_iter,
#     early_stopping = early_stopping
    )

clf.fit(train_X, train_y1)

SAVE_TRAINED_DATA_PATH = 'train1.learn'
# 学習結果を出力
joblib.dump(clf, SAVE_TRAINED_DATA_PATH)

# 学習済ファイルのロード
clf1 = joblib.load(SAVE_TRAINED_DATA_PATH)



# 検証用の説明変数
# test_X = test['bone_length', 'rotting_flesh', 'hair_length', 'has_soul', 'color_black','color_blood','color_blue','color_clear','color_green','color_white']
test_X1 = testing[x]

# 学習結果の検証
predict_y1 = clf1.predict_proba(test_X1)
predict_Y1 = clf1.predict(test_X1)
print(predict_Y1)

# 学習データの精度
print("学習データの精度: {:.3f}".format(clf.score(train_X, train_y1)))

# テスト結果の精度
print("テスト結果の精度: {:.3f}".format(clf.score(test_X1, predict_Y1)))

In [ ]:
# 提出用データ
sub = pd.read_csv('sample_submission.csv')
sub['type'] = list(predict_Y1)
sub.to_csv('sample_submission.csv', index=False)